<a href="https://colab.research.google.com/github/talhaanwarch/Corona_Virus/blob/master/Corona_albumentations.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
#https://arxiv.org/ftp/arxiv/papers/2003/2003.13145.pdf

In [3]:
!git clone https://github.com/mjkvaak/ImageDataAugmentor.git

Cloning into 'ImageDataAugmentor'...
remote: Enumerating objects: 98, done.
remote: Counting objects: 100% (98/98), done.
remote: Compressing objects: 100% (81/81), done.
remote: Total 263 (delta 54), reused 38 (delta 15), pack-reused 165
Receiving objects: 100% (263/263), 130.33 KiB | 766.00 KiB/s, done.
Resolving deltas: 100% (153/153), done.


In [4]:
from ImageDataAugmentor.image_data_augmentor import *

Using TensorFlow backend.


In [5]:
#Setup Kaggle DIR; copy json file ; chmod
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json
!ls ~/.kaggle

kaggle.json


In [6]:

#Install kaggle packages
!pip install -q kaggle
!pip install -q kaggle-cli

     |████████████████████████████████| 81kB 6.3MB/s 
     |████████████████████████████████| 5.3MB 31.8MB/s 
     |████████████████████████████████| 112kB 60.8MB/s 
     |████████████████████████████████| 51kB 10.0MB/s 
     |████████████████████████████████| 112kB 68.6MB/s 


In [7]:
#!kaggle datasets list
!kaggle datasets list -s tawsifurrahman/covid19-radiography-database

ref                                          title                          size  lastUpdated          downloadCount  
-------------------------------------------  -----------------------------  ----  -------------------  -------------  
tawsifurrahman/covid19-radiography-database  COVID-19 Radiography Database   1GB  2020-03-28 23:36:00            120  


In [8]:
!kaggle datasets download -d tawsifurrahman/covid19-radiography-database

 98% 1.13G/1.15G [00:15<00:00, 72.5MB/s]
100% 1.15G/1.15G [00:15<00:00, 78.9MB/s]


In [0]:
%%capture
!unzip covid19-radiography-database.zip

In [0]:
%%capture
!pip install efficientnet

In [0]:
import glob
import pandas as pd
import os
import numpy as np

In [0]:

covid=os.listdir('/content/COVID-19 Radiography Database/COVID-19')
normal=os.listdir('/content/COVID-19 Radiography Database/NORMAL')
pneumonia=os.listdir('/content/COVID-19 Radiography Database/Viral Pneumonia')


In [13]:
cov=list(zip(covid,['covid']*(len(covid))))
covid_df = pd.DataFrame(cov,columns=['file','label']) 
covid_df.head()

,file,label
0,COVID-19 (48).png,covid
1,COVID-19(178).png,covid
2,COVID-19(180).png,covid
3,COVID-19 (19).png,covid
4,COVID-19(155).png,covid


In [14]:
norm=list(zip(normal,['normal']*(len(normal))))
normal_df = pd.DataFrame(norm,columns=['file','label']) 
normal_df.head()

,file,label
0,NORMAL (110).png,normal
1,NORMAL (360).png,normal
2,NORMAL (131).png,normal
3,NORMAL (662).png,normal
4,NORMAL (1009).png,normal


In [15]:
pneumonia=list(zip(pneumonia,['pneumonia']*(len(pneumonia))*2))
pneumonia_df = pd.DataFrame(pneumonia,columns=['file','label']) 
pneumonia_df.head()

,file,label
0,Viral Pneumonia (1067).png,pneumonia
1,Viral Pneumonia (1094).png,pneumonia
2,Viral Pneumonia (1287).png,pneumonia
3,Viral Pneumonia (1218).png,pneumonia
4,Viral Pneumonia (876).png,pneumonia


In [16]:
dataframe=pd.concat([covid_df,normal_df,pneumonia_df])
dataframe.head()

,file,label
0,COVID-19 (48).png,covid
1,COVID-19(178).png,covid
2,COVID-19(180).png,covid
3,COVID-19 (19).png,covid
4,COVID-19(155).png,covid


In [17]:
df = pd.get_dummies(dataframe['label'])
df.head()

,covid,normal,pneumonia
0,1,0,0
1,1,0,0
2,1,0,0
3,1,0,0
4,1,0,0


In [18]:
df = pd.concat([dataframe, df], axis=1)
df.head()

,file,label,covid,normal,pneumonia
0,COVID-19 (48).png,covid,1,0,0
1,COVID-19(178).png,covid,1,0,0
2,COVID-19(180).png,covid,1,0,0
3,COVID-19 (19).png,covid,1,0,0
4,COVID-19(155).png,covid,1,0,0


In [0]:
df=df.sample(frac=1)

In [0]:
!mkdir data

In [0]:
!cp -r /content/COVID-19\ Radiography\ Database/COVID-19/* data


In [0]:
!cp -r /content/COVID-19\ Radiography\ Database/NORMAL/* data


In [0]:
!cp -r /content/COVID-19\ Radiography\ Database/Viral\ Pneumonia/* data


In [0]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
from albumentations import *


In [0]:
aug=Compose([
        RandomRotate90(),
        VerticalFlip(),
        HorizontalFlip(),
        Transpose(),
        RandomScale(),
        OneOf([
            IAAAdditiveGaussianNoise(),
            GaussNoise(),
            GaussianBlur(),
        ], p=0.3),
        OneOf([
            MotionBlur(p=.2),
            MedianBlur(blur_limit=3, p=.1),
            Blur(blur_limit=3, p=.1),
        ], p=0.2),
        ShiftScaleRotate(shift_limit=0.0625, scale_limit=0.2, rotate_limit=45, p=.2),
        OneOf([
            OpticalDistortion(p=0.3),
            GridDistortion(p=.1),
            IAAPiecewiseAffine(p=0.3),
        ], p=0.2),
        OneOf([
            CLAHE(clip_limit=2),
            IAASharpen(),
            IAAEmboss(),
            RandomContrast(),
            RandomBrightness(),
        ], p=0.3),
        #HueSaturationValue(p=0.3),
    ], p=1)

In [0]:
#from keras_preprocessing.image import ImageDataGenerator
from ImageDataAugmentor.image_data_augmentor import *
data_gen= ImageDataAugmentor(
    #horizontal_flip=True,
    #vertical_flip=True,
    #rotation_range=90,
    # width_shift_range=0.2,
    # height_shift_range=0.2,
    # zoom_range=.2,
    rescale=1/255,
    # fill_mode='nearest',
    # shear_range=0.2,
    # brightness_range=[0.2, 2.0],
    augment =aug)

In [0]:
img_shape=300
batch_size=16

In [0]:
from tensorflow.keras.models import Sequential,Model
from tensorflow.keras.layers import Activation, Dropout, Flatten, Dense, Conv2D, MaxPool2D, BatchNormalization,Input,MaxPooling2D,GlobalMaxPooling2D,concatenate
from tensorflow.keras.layers import GlobalAveragePooling2D
from tensorflow.keras.applications.resnet50 import ResNet50
from tensorflow.keras.callbacks import ModelCheckpoint,ReduceLROnPlateau
from tensorflow.keras.optimizers import Adam
import tensorflow.keras.backend as K
import tensorflow as tf

In [0]:
from sklearn.model_selection import KFold
import gc

from sklearn.metrics import classification_report,accuracy_score,f1_score
classification_reports=[]
kf = KFold(n_splits=5)
history=[]
accuracy=[]
f1_scores=[]
fold=0
for train_index, test_index in kf.split(df):
  train=df.iloc[train_index,:]
  val=df.iloc[test_index,:]
  print('-----------fold {}--------------'.format(fold))
  train_generator=data_gen.flow_from_dataframe(train,directory='data',
                                                      target_size=(img_shape,img_shape),
                                                      x_col="file",
                                                      y_col=['covid','normal','pneumonia'],
                                                      class_mode='raw',
                                                      shuffle=True,
                                                      batch_size=batch_size)

  val_generator=data_gen.flow_from_dataframe(val,directory='data',
                                                        target_size=(img_shape,img_shape),
                                                        x_col="file",
                                                        y_col=['covid','normal','pneumonia'],
                                                        class_mode='raw',
                                                        shuffle=False,
                                                        batch_size=batch_size)
  #train_generator.next()[0].shape,train_generator.next()[1].shape
  import efficientnet.tfkeras as efn
  model =efn.EfficientNetB4(weights ='noisy-student', include_top=False, input_shape = (img_shape,img_shape,3))
    
  x = model.output
  x = GlobalAveragePooling2D()(x)
  x = Dropout(0.3)(x)
  x = Dense(128, activation="relu")(x)
  x = Dropout(0.3)(x)
  x = Dense(64, activation="relu")(x)
  predictions = Dense(3, activation="softmax")(x)
  model = Model(inputs=model.input, outputs=predictions)

  def custom_loss(y_true, y_pred):
      return tf.keras.losses.categorical_crossentropy(y_true, y_pred, label_smoothing=0.1)
  model.compile(optimizer=Adam(0.0001), loss=custom_loss, metrics=['accuracy'])

  results = model.fit(train_generator,epochs=15,
                              steps_per_epoch=train_generator.n/batch_size,
                              validation_data=val_generator,
                             validation_steps=val_generator.n/batch_size,
                              callbacks=[ReduceLROnPlateau(monitor='val_loss', factor=0.3,patience=2, min_lr=0.000001)])
  val_generator.reset()
  y_pred=model.predict(val_generator,steps=val_generator.n/batch_size,verbose=1)
  y_pred=y_pred.round().astype(int)
  y_true=val.iloc[:,2::]
  classification_reports.append(classification_report(y_true, y_pred,target_names=['covid','normal','pneumonia']))#
  accuracy.append(accuracy_score(y_true,y_pred))
  f1=f1_score(y_true,y_pred,average='macro')
  print('f1 score is ', f1)
  f1_scores.append(f1)
  fold+=1
  del model
  tf.keras.backend.clear_session()
  gc.collect()
  

-----------fold 0--------------
Found 2324 validated image filenames.
Found 581 validated image filenames.
71680000/71678424 [==============================] - 3s 0us/step
Epoch 1/15
146/145 [==============================] - 131s 894ms/step - loss: 0.7327 - accuracy: 0.7401 - val_loss: 0.8738 - val_accuracy: 0.6334 - lr: 1.0000e-04
Epoch 2/15
146/145 [==============================] - 130s 889ms/step - loss: 0.5085 - accuracy: 0.9010 - val_loss: 0.6122 - val_accuracy: 0.8296 - lr: 1.0000e-04
Epoch 3/15
146/145 [==============================] - 128s 879ms/step - loss: 0.4676 - accuracy: 0.9161 - val_loss: 0.4657 - val_accuracy: 0.9122 - lr: 1.0000e-04
Epoch 4/15
146/145 [==============================] - 133s 911ms/step - loss: 0.4420 - accuracy: 0.9264 - val_loss: 0.3732 - val_accuracy: 0.9604 - lr: 1.0000e-04
Epoch 5/15
146/145 [==============================] - 130s 891ms/step - loss: 0.4230 - accuracy: 0.9346 - val_loss: 0.3762 - val_accuracy: 0.9621 - lr: 1.0000e-04
Epoch 6/15
14

In [0]:
classification_reports[0]

'              precision    recall  f1-score   support\n\n       covid       0.97      0.97      0.97        35\n      normal       0.98      0.99      0.98       276\n   pneumonia       0.99      0.98      0.98       270\n\n   micro avg       0.98      0.98      0.98       581\n   macro avg       0.98      0.98      0.98       581\nweighted avg       0.98      0.98      0.98       581\n samples avg       0.98      0.98      0.98       581\n'